In [1]:
from transformers import Trainer, TrainingArguments, TrainerCallback
from model import get_model, TweetDataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import pickle, torch
import sys,os
sys.path.append('../')

In [ ]:
import copy
class CustomCallback(TrainerCallback):
    
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer
    
    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = copy.deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy

In [6]:
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


def load_split_data(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)
    
def train():
    current_dir = '/Users/arya/Desktop/mlds-proj/tweet-sentiment-analysis/src/'
    file_path = os.path.join(current_dir, '..', 'data', 'processed', 'train_test_0.2_split.pkl')

    print(f'[INFO]\tExtracting data from {file_path}...')
    train_texts, test_texts, train_labels, test_labels = load_split_data(file_path)
    # print(test_texts, test_labels)
    train_dataset = TweetDataset(train_texts, train_labels)
    test_dataset = TweetDataset(test_texts, test_labels)


    print('[INFO]\tInitializing model...')
    model = get_model()
    training_args = TrainingArguments(
        output_dir='../results/models',
        num_train_epochs=2,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='../results/logs',
        logging_steps=10,
        evaluation_strategy="epoch"
    )
    
    print('[INFO]\tTraining model...')
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
    )
    trainer.add_callback(CustomCallback(trainer))
    trainer.train()


    print('[INFO]\tSaving model...')
    save_path = os.path.join(current_dir, '..', 'results', 'models', 'bert-finetuned')
    trainer.save_model(save_path)
    print(f'[INFO]\tModel saved at {save_path}.')

    print('[INFO]\tEvaluating model...')
    trainer.evaluate()
    print('[INFO]\tDone.')

    return trainer

In [7]:

trainer = train()

[INFO]	Extracting data from /Users/arya/Desktop/mlds-proj/tweet-sentiment-analysis/src/../data/processed/train_test_0.2_split.pkl...
[INFO]	Initializing model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/arya/miniconda3/lib/python3.11/site-packages/accelerate/accelerator.py:443: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


[INFO]	Training model...


  0%|          | 0/400 [00:00<?, ?it/s]

{'loss': 0.7702, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}
{'loss': 0.8009, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.1}
{'loss': 0.6558, 'learning_rate': 3e-06, 'epoch': 0.15}
{'loss': 0.6577, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.2}
{'loss': 0.7613, 'learning_rate': 5e-06, 'epoch': 0.25}
{'loss': 0.6979, 'learning_rate': 6e-06, 'epoch': 0.3}
{'loss': 0.6471, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.35}
{'loss': 0.6576, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.4}
{'loss': 0.6279, 'learning_rate': 9e-06, 'epoch': 0.45}
{'loss': 0.6578, 'learning_rate': 1e-05, 'epoch': 0.5}
{'loss': 0.6247, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.55}
{'loss': 0.5954, 'learning_rate': 1.2e-05, 'epoch': 0.6}
{'loss': 0.5465, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.65}
{'loss': 0.6927, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.7}
{'loss': 0.4886, 'learning_rate': 1.5e-05, 'epoch': 0.75}
{'loss': 0.5337, 'learning_rat

...

{'train_loss': 0.23620183765888214, 'train_accuracy': 0.913125, 'train_f1': 0.9178001182732111, 'train_precision': 0.8680089485458613, 'train_recall': 0.973651191969887, 'train_runtime': 24.7713, 'train_samples_per_second': 64.591, 'train_steps_per_second': 4.037, 'epoch': 2.0}


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.5221846699714661, 'eval_accuracy': 0.7675, 'eval_f1': 0.7973856209150327, 'eval_precision': 0.71484375, 'eval_recall': 0.9014778325123153, 'eval_runtime': 7.397, 'eval_samples_per_second': 54.076, 'eval_steps_per_second': 3.38, 'epoch': 2.0}
{'train_runtime': 308.4969, 'train_samples_per_second': 10.373, 'train_steps_per_second': 1.297, 'train_loss': 0.538849510550499, 'epoch': 2.0}
[INFO]	Saving model...
[INFO]	Model saved at /Users/arya/Desktop/mlds-proj/tweet-sentiment-analysis/src/../results/models/bert-finetuned.
[INFO]	Evaluating model...


  0%|          | 0/25 [00:00<?, ?it/s]

[INFO]	Done.


In [8]:
train_dict={}

for i in trainer.state.log_history:
    train_dict[i['step']]=i

train_dict

{10: {'loss': 0.7702,
  'learning_rate': 1.0000000000000002e-06,
  'epoch': 0.05,
  'step': 10},
 20: {'loss': 0.8009,
  'learning_rate': 2.0000000000000003e-06,
  'epoch': 0.1,
  'step': 20},
 30: {'loss': 0.6558, 'learning_rate': 3e-06, 'epoch': 0.15, 'step': 30},
 40: {'loss': 0.6577,
  'learning_rate': 4.000000000000001e-06,
  'epoch': 0.2,
  'step': 40},
 50: {'loss': 0.7613, 'learning_rate': 5e-06, 'epoch': 0.25, 'step': 50},
 60: {'loss': 0.6979, 'learning_rate': 6e-06, 'epoch': 0.3, 'step': 60},
 70: {'loss': 0.6471,
  'learning_rate': 7.000000000000001e-06,
  'epoch': 0.35,
  'step': 70},
 80: {'loss': 0.6576,
  'learning_rate': 8.000000000000001e-06,
  'epoch': 0.4,
  'step': 80},
 90: {'loss': 0.6279, 'learning_rate': 9e-06, 'epoch': 0.45, 'step': 90},
 100: {'loss': 0.6578, 'learning_rate': 1e-05, 'epoch': 0.5, 'step': 100},
 110: {'loss': 0.6247,
  'learning_rate': 1.1000000000000001e-05,
  'epoch': 0.55,
  'step': 110},
 120: {'loss': 0.5954, 'learning_rate': 1.2e-05, 'ep

In [10]:
# testing
torch.save(train_dict, os.path.join('.', '..', 'results', 'logs', 'train_loss.pt'))